In [1]:
from scipy.optimize import linprog
import numpy as np

# Shifts starting on the different days of the week
shift = ['sSun', 'sMon', 'sTue', 'sWed', 'sThu', 'sFri', 'sSat']

# Actual days of the week
days = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']

# Demand for workers on different days of the week
b = np.array([3, 2, 2, 2, 6, 7, 7])

# Unit cost of workers on the different shifts
c = np.array([1, 1, 1, 1, 1, 1, 1])

# Matrix for shifts and the days they work
A = np.array([
    [1, 1, 1, 1, 0, 0, 0],
    [0, 1, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1],
    [1, 0, 0, 0, 1, 1, 1],
    [1, 1, 0, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 1]
])

# Solve the linear programming problem
result = linprog(c, A_ub=-A, b_ub=-b, method='highs')

# Print the solution
print("Optimal solution:", result.x, "Objective = ", result.fun)

A@result.x, b

Optimal solution: [4. 1. 1. 0. 0. 1. 1.] Objective =  8.0


(array([6., 2., 2., 2., 6., 7., 7.]), array([3, 2, 2, 2, 6, 7, 7]))

### The Pizzerias worker shift problem

The model for the Pizzeria problem is described in Python using GurobiPy.


First we need to get install gurobipy using pip:

In [2]:
!pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 5.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


**Vanilla model**

Now we set up the model, solve and display the result as done in the learnOR tutorial using the linprog package in R.

The model is to minimize the number of workers working the different shifts ($S$):

$$\min \sum_{s \in S} x_{s}$$

where $x_s$ denoted the number of workers on work shift, $s\in S$. On which day, $d\in D$, the different shifts are determined by the data matrix $A$ (binary indicator matrix) and so we have the condition

$$\sum_{s \in S} A_{s,d} x_{s} \ge b_d \quad \forall d \in D$$

where the parameter $b_d \ge 0$ tells us the minimum required number of workers needed for that day $d$.

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Create a new model
model = gp.Model("Pizzeria")

# Create the set of objects we are working with
Days = list(range(7)) # 'sun' = 0, 'mon' = 1, ..., 'sat' = 6
Shifts = Days # ['shift-' + d for d in Days]

# Create continuous decision variable for number of workers required for given day
x = model.addVars(Days)

# Set objective number of workers, note each shift is 4 days!
model.setObjective(gp.quicksum(x[s] for s in Shifts), GRB.MINIMIZE)

# Add constraints for the minimal workers required
A = np.array([[1.,1,1,1,0,0,0],
      [0,1,1,1,1,0,0],
      [0,0,1,1,1,1,0],
      [0,0,0,1,1,1,1],
      [1,0,0,0,1,1,1],
      [1,1,0,0,0,1,1],
      [1,1,1,0,0,0,1]]) # Data mxn
b = [3,2,2,2,6,7,7]     # Demand data of length m (number of days)
model.addConstrs( gp.quicksum(x[s] * A[d,s] for s in Shifts) >= b[d] for d in Days)

# Optimize model
model.optimize()

# Display the decision and its objective value
xnp = np.array([x[s].X for s in Shifts])
print("x = ", xnp)
print('Objective = %g' % model.objVal)
A@xnp, b

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 23.6.0 23H222)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 7 columns and 28 nonzeros
Model fingerprint: 0xa9b8e7d4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 7e+00]
Presolve time: 0.04s
Presolved: 7 rows, 7 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.900000e+01   0.000000e+00      0s
       5    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.11 seconds (0.00 work units)
Optimal objective  8.000000000e+00
x =  [4. 1. 1. 0. 0. 1. 1.]
Objective = 8


(array([6., 2., 2., 2., 6., 7., 7.]), [3, 2, 2, 2, 6, 7, 7])